# R3ACT: Resilience, Reaction and Recovery Analysis of Critical Transitions

## SkillCorner X PySport Analytics Cup - Research Track

This notebook implements the R3ACT system to objectively measure individual and collective response capabilities to critical errors in elite football.


## 1. Introduction

R3ACT analyzes the resilience, reaction, and recovery of players and teams following critical events using tracking data and enriched events. The system calculates three main metrics:

- **CRT (Cognitive Reset Time)**: Individual physical/cognitive recovery time
- **TSI (Team Support Index)**: Collective team response (proximity, possession, structure)
- **GIRI (Goal Impact Response Index)**: Tactical change post-goal

Data is loaded directly from the SkillCorner Open Data repository.


In [ ]:
# Install dependencies (run only if necessary)
# !pip install pandas numpy scipy requests streamlit plotly


In [ ]:
# Import R3ACT system modules
from src.r3act_system import R3ACTSystem
import pandas as pd
import numpy as np


## 2. Methods

### 2.1 Critical Event Detection

The system automatically detects critical events from `dynamic_events.csv`:
- Possession losses (classified by field zone)
- Failed passes (with danger context)
- Goals (scored and conceded)
- Defensive errors
- Conceded interceptions

Each event has a configurable weight allowing prioritization based on tactical importance.

### 2.2 Baseline Calculation

The baseline is calculated as an average across **all matches** (10 matches), not per individual match. This enables consistent and scalable comparisons.

### 2.3 R3ACT Metrics

**CRT**: Uses Mahalanobis distance + EWMA to identify recovery. Compares post-error physical metrics (velocity, position, distance) with player baseline.

**TSI**: Combines three components:
- Physical proximity of team to error-committing player
- Change in possession frequency
- Defensive structure changes (compactness)

**GIRI**: Measures tactical changes post-goal (block height, velocity, compactness).

### 2.4 Temporal Windows

The system supports configurable windows (2, 5, 10 minutes). Pre and post are symmetric for fair comparison.


In [ ]:
# Initialize R3ACT system
# Temporal window: 'short' (2min), 'medium' (5min), 'long' (10min)
r3act = R3ACTSystem(time_window='medium')

# Process all matches
# Note: load_tracking=True loads complete data (slower but precise)
# For quick testing, use load_tracking=False
results_df = r3act.process_all_matches(load_tracking=True)


## 3. Results

The final dataset contains R3ACT metrics calculated for each detected critical event.


In [ ]:
# Explore results
print(f"Total critical events analyzed: {len(results_df)}")
print(f"\nDistribution by event type:")
print(results_df['event_type'].value_counts())

# Metric statistics
if 'CRT' in results_df.columns:
    print(f"\nCRT - Average recovery time: {results_df['CRT'].mean():.2f} seconds")
    print(f"CRT - Median: {results_df['CRT'].median():.2f} seconds")

if 'TSI' in results_df.columns:
    print(f"\nTSI - Average team support index: {results_df['TSI'].mean():.3f}")
    print(f"TSI - Median: {results_df['TSI'].median():.3f}")

if 'GIRI' in results_df.columns:
    giri_events = results_df[results_df['GIRI'].notna()]
    if len(giri_events) > 0:
        print(f"\nGIRI - Average post-goal response: {giri_events['GIRI'].mean():.3f}")
        print(f"GIRI - Goal events: {len(giri_events)}")


In [ ]:
# Complete statistical summary
summary = r3act.get_results_summary()
print("R3ACT Analysis Summary:")
for key, value in summary.items():
    print(f"{key}: {value}")


## 4. Conclusion

The R3ACT system provides objective and scalable metrics for analyzing resilience to critical errors. The metrics are parametrizable, allowing adaptation to different tactical philosophies and analysis needs.

**Limitations**: Calculation requires complete tracking data. Some events may not have calculated metrics if tracking or phases_of_play data is missing.

**Future improvements**: Include more critical event types, additional pressure metrics, and more sophisticated temporal pattern analysis.
 